In [ ]:
import os
import pickle
import sys
import time
from copy import deepcopy
from pathlib import Path
from typing import Dict, Sequence, Tuple, Union

import numpy as np
import torch
import wfdb
from torch.nn.parallel import DataParallel as DP
from torch.nn.parallel import DistributedDataParallel as DDP  # noqa: F401
from torch_ecg.cfg import CFG, DEFAULTS
from torch_ecg.utils.misc import str2bool
from torch_ecg.utils.utils_nn import default_collate_fn as collate_fn
from tqdm.auto import tqdm

from cfg import ModelCfg, TrainCfg
from data_reader import CODE15, PTBXL, SamiTrop
from dataset import CINC2025Dataset
from models import CRNN_CINC2025, FM_CINC2025
from outputs import CINC2025Outputs
from trainer import CINC2025Trainer
from utils.samplers import BalancedBatchSampler

# sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/")
# sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/bib_lookup/")

%load_ext autoreload
%autoreload 2

## Train models

In [ ]:
db_dir = Path("/home/wenh06/Jupyter/Hot-data/cinc2025/")
# db_dir = Path("/home/wenh06/Jupyter/Hot-data/cinc2025-test/")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
TEST_FLAG = False

if ModelCfg.torch_dtype == torch.float64:
    torch.set_default_tensor_type(torch.DoubleTensor)
    DTYPE = np.float64
else:
    DTYPE = np.float32

CINC2025Dataset.__DEBUG__ = False
CRNN_CINC2025.__DEBUG__ = False
FM_CINC2025.__DEBUG__ = False
CINC2025Trainer.__DEBUG__ = False

In [ ]:
repeats = 4
# for subsample in [0.01, 0.1, 0.5, 1.0]:
for subsample in [0.01]:
    for _ in range(repeats):
        train_config = deepcopy(TrainCfg)
        train_config.db_dir = db_dir
        train_config.debug = True

        train_config.n_epochs = 20
        train_config.batch_size = 192  # 16G (Tesla T4)
        # train_config.log_step = 20

        # for CRNN only, comment if using FM_CINC2025
        train_config.learning_rate = 3e-4  # 5e-4, 1e-3
        train_config.lr = train_config.learning_rate
        train_config.max_lr = 9e-4

        train_config.keep_checkpoint_max = 0

        train_config.lr_scheduler = "one_cycle"
        train_config.early_stopping.patience = train_config.n_epochs // 3

        train_config.extra_experiment = True
        train_config.subsample = subsample  # 0.01, 0.10, 0.5, 1.0

        # augmentations configurations
        # TODO: add augmentation configs

        model_config = deepcopy(ModelCfg)
        # model_config

        model_config.crnn.dem_encoder.mode = "film"
        model = CRNN_CINC2025(config=model_config.crnn)
        # model_config.fm.backbone_cache_dir = "/home/wenh06/Jupyter/models/ST-MEM/st_mem_vit_base_encoder.pth"
        # model = FM_CINC2025(config=model_config.fm)

        model = model.to(device=DEVICE)
        if isinstance(model, DP):
            print("model size:", model.module.module_size, model.module.module_size_)
            print("Using devices:", model.device_ids)
        else:
            print("model size:", model.module_size, model.module_size_)
            print("Using device:", model.device)

        ds_train = CINC2025Dataset(train_config, training=True, lazy=True)
        ds_val = CINC2025Dataset(train_config, training=False, lazy=True)

        if isinstance(model, FM_CINC2025):
            print("Using FM_CINC2025 model, adjusting fs and input_len")
            ds_train.reset_resample_fs(model_config.fm.fs[model_config.fm.name], reload=False)
            ds_train.reset_input_len(model_config.fm.input_len[model_config.fm.name], reload=False)
            ds_val.reset_resample_fs(model_config.fm.fs[model_config.fm.name], reload=False)
            ds_val.reset_input_len(model_config.fm.input_len[model_config.fm.name], reload=False)

        trainer = CINC2025Trainer(
            model=model,
            model_config=model_config,
            train_config=train_config,
            device=DEVICE,
            lazy=True,
        )
        trainer._setup_dataloaders(ds_train, ds_val)
        best_model_state_dict = trainer.train()

        trainer.log_manager.flush()
        trainer.log_manager.close()

        del trainer, model, best_model_state_dict, ds_train, ds_val
        torch.cuda.empty_cache()

        time.sleep(10)

## Results analyze and visualize

In [ ]:
df_results = dict()

In [ ]:
df_results["resnet-nc"] = pd.read_csv(
    "./results/TorchECG_02-21_17-38_CRNN_CINC2025_resnet_nature_comm_bottle_neck_adamw_amsgrad_LR_0.0001_BS_128.csv"
)
df_results["resnet-nc-se"] = pd.read_csv(
    "./results/TorchECG_02-22_01-33_CRNN_CINC2025_resnet_nature_comm_bottle_neck_se_adamw_amsgrad_LR_0.0001_BS_128.csv"
)
df_results["tresnet-m"] = pd.read_csv(
    "./results/TorchECG_02-22_09-09_CRNN_CINC2025_tresnetM_adamw_amsgrad_LR_0.0001_BS_128.csv"
)
df_results["tresnet-n"] = pd.read_csv(
    "./results/TorchECG_02-22_01-38_CRNN_CINC2025_tresnetN_adamw_amsgrad_LR_0.0001_BS_128.csv"
)
df_results["tresnet-f"] = pd.read_csv(
    "./results/TorchECG_02-22_09-13_CRNN_CINC2025_tresnetF_adamw_amsgrad_LR_0.0001_BS_128.csv"
)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

part = "train"
metric = "challenge_score"

for k, df in df_results.items():
    df_metric = df[df.part == part][[metric, "epoch"]].dropna()
    ax.plot(df_metric.epoch, df_metric[metric], label=k)
ax.legend(loc="best")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

part = "val"
metric = "challenge_score"

for k, df in df_results.items():
    df_metric = df[df.part == part][[metric, "epoch"]].dropna()
    ax.plot(df_metric.epoch, df_metric[metric], label=k)
ax.legend(loc="best")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

part = "train"
metric = "chagas_f_measure"

for k, df in df_results.items():
    df_metric = df[df.part == part][[metric, "epoch"]].dropna()
    ax.plot(df_metric.epoch, df_metric[metric], label=k)
ax.legend(loc="best")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

part = "val"
metric = "chagas_f_measure"

for k, df in df_results.items():
    df_metric = df[df.part == part][[metric, "epoch"]].dropna()
    ax.plot(df_metric.epoch, df_metric[metric], label=k)
ax.legend(loc="best")